In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

In [2]:
metadata_df = pd.read_csv("data/network_metadata.tab", sep="\t")
metadata_df.loc[:, "IMDB_id"] = metadata_df["IMDB_id"].str[2:].astype('int')
# metadata_df.head()

,GexfID,Title,IMDB_id,ReleaseDate,Slug,WeightedDegree,Modularity,PathLength,Diameter,ClusteringCoefficient,ConnectedComponents,Density,Edges,Characters
0,1,10 Things I Hate About You,147800,1999,10-Things-I-Hate-About-You,"13,040","0,194","1,779","3,000","0,567","3,000","0,233",225,25
1,2,12,488478,2007,12,"4,227","0,517","2,170","3,000","0,610","6,000","0,077",102,42
2,3,Twelve and Holding,417385,2005,Twelve-and-Holding,"7,333","0,473","2,449","4,000","0,593","2,000","0,140",139,26
3,5,127 Hours,1542344,2010,127-Hours,"3,833","0,160","1,722","2,000","0,339","4,000","0,152",33,11
4,6,1492: Conquest of Paradise,103594,1992,1492:-Conquest-of-Paradise,"7,941","0,350","2,043","4,000","0,705","1,000","0,153",164,34


In [3]:
imdb_data_df = pd.read_json("data/gexf_imdb_metadata.json", orient='index')

In [4]:
df = pd.merge(metadata_df, imdb_data_df, left_on="IMDB_id", right_on="movie_id")
df.loc[:,'directors-writers'] = df.loc[:,'directors'] + df.loc[:,'writers']
for i in range(len(df)):
    df.loc[:,'directors-writers'][i] = list(set(df.loc[:,'directors-writers'][i]))

C:\Users\Robin McCann\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [41]:
# Load Movies Graph and relabel nodes
Movies_Graph = nx.read_adjlist("data/movies_graph.adjlist")
mapping = {}
for node in Movies_Graph:
    mapping[node] = str(node).replace("_", " ")


Movies_Graph=nx.relabel_nodes(Movies_Graph, mapping)

In [42]:
# Load Writers Graph and relabel nodes
Writers_Graph = nx.read_adjlist("data/writers_graph.adjlist")
mapping = {}
for node in Writers_Graph:
    mapping[node] = str(node).replace("_", " ")


Writers_Graph=nx.relabel_nodes(Writers_Graph, mapping)

In [26]:
# Writers ordered by most connections
all_writers = list(Writers_Graph.nodes())
all_writers_sorted = sorted(all_writers, key=lambda x: Writers_Graph.degree(x), reverse=True)
'''
for i in range(20):
    print(f"{all_writers_sorted[i]}, degree: {Writers_Graph.degree(all_writers_sorted[i])}")
'''

'\nfor i in range(20):\n    print(f"{all_writers_sorted[i]}, degree: {Writers_Graph.degree(all_writers_sorted[i])}")\n'

In [25]:
# Movies ordered by most connections
all_movies = list(Movies_Graph.nodes())
all_movies_sorted = sorted(all_movies, key=lambda x: Movies_Graph.degree(x), reverse=True)                  
'''
for i in range(20):
    print(f"{all_movies_sorted[i]}, degree: {Movies_Graph.degree(all_movies_sorted[i])}")
'''

'\nfor i in range(20):\n    print(f"{all_movies_sorted[i]}, degree: {Movies_Graph.degree(all_movies_sorted[i])}")\n'

In [68]:
# Find communities among movies
movie_communities = sorted(list(nx.algorithms.community.modularity_max.greedy_modularity_communities(Movies_Graph)),
                          key=lambda x: len(x), reverse=True)
len(movie_communities)

273

In [59]:
# Find communities among writers
writer_communities = sorted(list(nx.algorithms.community.modularity_max.greedy_modularity_communities(Writers_Graph)), 
                     key=lambda x: len(x), reverse=True)
len(writer_communities)

265

In [ ]:
# Onion Decomposition